# Compute the decay cascade of $1s \to 3p$ photoexcited neon

In [1]:
using JAC

Most excited atomic levels decay rapidly with lifetimes typically ranging from the femto- to the (meta-stable) micro-second regime; more often than not, this decay *proceeds* via many intermediate levels before the state of the atom or ion becomes stable. More specifically, here we shall talk about *atomic cascades* if (i) three or more charges states are involved in the excitation and/or decay cascade and (ii) if it becomes cumbersome to specify all the intermediate and final levels explictly. For such cascades, moreover, the observed ion distributions or the x-ray and electron spectra are usually much more interesting than the individual decay pathes or which intermediate levels contribute to the particular decay.

To model such cascades theoretically, one requires of course both, the computation of a reasonably good -- excitation and decay -- data set as well as the proper *combination* of these data in order to make the theoretical results comparable to observations. In JAC, we therefore distinguish *two separate steps for all kinds of cascade computations:* **(1) The computation of the cascade tree**, i.e. of all transition amplitudes, rates and cross sections that appear in any physically allowed *decay path* of the atoms. Here, a *path* refers to a sequence of atomic transitions which either (explicitly) excite the atoms due to photoionization or electron impact, or which subsequently leads to the stabilization of the atom. Afterwards, **(2) a cascade simulation** need to be performed with the data from the cascade tree of step (1) in order to derive the physically requested information.

In practice, of course, *atomic cascades* may become very complex and unfeasible to compute. Therefore, in order to deal with such cascades, one often needs to restrict either the number of charge states, the atomic processes to be included into the cascade, the approach for describing the required wave functions, or several others features. The usual goal is to generate and process all amplitudes for the given *physical processes* (e.g. photon emission, Auger, photoionization, ...) *automatically*, while restricting the comptutions by means of some predefined approach or a few additional assumptions.


As a very first example of an atomic cascde, we shall consider the decay of photo-excited $1s \to 3p$ neon which either decays by autoionization of photon emission. In this example, we shall restrict ourselves to a very simple approach in generating all the necessary wave functions as well as to the simulation of the final ion and level distributions. Further properties, which can be simulated from the cascade tree data, will likely be considered in the future.


In JAC, the (transition) data of a cascade tree is computed be means of a `Cascade.Computation()`, a container to keep all necessary information about the atom, its initial state, the maximum number of emitted electrons and other information as easily derived from the experimental set-up. We will easily understand the necessary input for such a cascade tree computation by: 


In [2]:
? Cascade.Computation

`struct  Cascade.Computation`       ... defines a type for a cascade computation, i.e. for the computation of a whole photon excitation, photon ionization and/or          decay cascade. The – input and control – data from this computation can be modified, adapted and refined to the practical needs,          and before the actual computations are carried out explictly. Initially, this struct just contains the physical meta-data about the          cascade, that is to be calculated, but a new instance of the same Cascade.Computation gets later enlarged in course of the          computation in order to keep also wave functions, level multiplets, etc.

```
+ name               ::String                          ... A name for the cascade
+ nuclearModel       ::Nuclear.Model                   ... Model, charge and parameters of the nucleus.
+ grid               ::Radial.Grid                     ... The radial grid to be used for the computation.
+ asfSettings        ::AsfSettings                     ... Provides the settings for the SCF process.
+ scheme             ::Cascade.AbstractCascadeScheme   ... Scheme of the atomic cascade (photoionization, decay, ...)
+ approach           ::Cascade.AbstractCascadeApproach 
    ... Computational approach/model that is applied to generate and evaluate the cascade; possible approaches are: 
        {AverageSCA(), SCA(), ...}
+ initialConfs       ::Array{Configuration,1}          
    ... List of one or several configurations that contain the level(s) from which the cascade starts.
+ initialMultiplets  ::Array{Multiplet,1}              
    ... List of one or several (initial) multiplets; either initialConfs 'xor' initialMultiplets can  be specified 
        for a given cascade computation.
```

---

`Cascade.Computation()`  ... constructor for an 'default' instance of a Cascade.Computation.

---

`Cascade.Computation(comp::Cascade.Computation;`

```
        name=..,               nuclearModel=..,             grid=..,              asfSettings=..,     
        scheme=..,             approach=..,                 initialConfigs=..,    initialMultiplets=..)
        
... constructor for re-defining the computation::Cascade.Computation.
```


Apart from a user-defined name (string), the specification of the nucleus and the radial grid, here we need to specify either the initial configurations *(x)or* the initial multiplets whose levels *starts* the photoionization or decay cascade. Of course, only one, configurations or multiplets, can be given, and an error is issued if both are specified for the same computation. As usual, the `asfsettings` determine how the SCF and CI computations are performed internally, although the default is here often most appropriate.

Two central input parameters for all cascade computations refer to the `scheme::Cascade.AbstractCascadeScheme` and the `approach::Cascade.AbstractCascadeApproach`. The cascade `scheme` determines whether we consider an photoionization or a decay scheme for the initial levels, and other useful schemes (electron-impact, double ionization, ...) might be added in the future:

In [3]:
? Cascade.AbstractCascadeScheme

`abstract type Cascade.AbstractCascadeScheme`      ... defines an abstract type to distinguish different excitation, ionization and decay schemes of an atomic cascade; see also:

```
+ struct StepwiseDecayScheme       
    ... to represent a standard decay scheme, starting from the levels of one or several initial multiplets.
+ struct PhotonIonizationScheme    
    ... to represent a (prior) photoionization part of a cascade for a list of given photon energies.
+ struct PhotonExcitationScheme    
    ... to represent a (prior) photo-excitation part of a cascade for a specified set of excitations.
+ struct ElectronExcitationScheme  
    ... to represent a (prior) electron-impact excitation part of a cascade for given electron energies (not yet).
```


Below, we shall consider the **stepwise decay** of the neon $1s 2s^2 2p^6 3p$ configuration; for such a decay scheme, further input data need to be specified by:

In [4]:
? Cascade.StepwiseDecayScheme

`struct  Cascade.StepwiseDecayScheme  <:  Cascade.AbstractCascadeScheme`       ... a struct to represent (and generate) a mean-field orbital basis.

```
+ processes             ::Array{Basics.AbstractProcess,1} 
    ... List of the atomic processes that are supported and should be included into the cascade.
+ maxElectronLoss       ::Int64             
    ... (Maximum) Number of electrons in which the initial- and final-state configurations can differ from each other; 
        this also determines the maximal steps of any particular decay path.
+ chargeStateShifts     ::Dict{Int64,Float64} 
    ... (N => en) total energy shifts of all levels with N electrons; these shifts [in a.u.] help open/close decay 
        channels by simply shifting the total energies of all levels.
+ NoShakeDisplacements  ::Int64             
    ... Maximum number of electron displacements due to shake-up  or shake-down processes in any individual step of cascade.
+ shakeFromShells       ::Array{Shell,1}        ... List of shells from which shake transitions may occur.
+ shakeToShells         ::Array{Shell,1}        ... List of shells into which shake transitions may occur.
```

---

`Cascade.StepwiseDecayScheme()`  ... constructor for an 'default' instance of a Cascade.StepwiseDecayScheme.


As seen from this definition of `Cascade.StepwiseDecayScheme`, several features about shake-up and -down processes are already prepared for input but not yet supported internally. For the moment, the `maxElectronLoss` is relevant as it determines the (maximal) number of charge states as well as the atomic processes to be considered in the automatic generation of the cascade tree. Below, we shall consider `Auger` and `Radiative` photon emission processes and, indeed, only these processes are supported  at present.

The second and very relevant input parameter for a cascade tree computation is the `approach::Cascade.AbstractCascadeApproach` which determines the approach to compute all required intermedaite multiplets. Although there is not much freedom here at the moment, it is obvious that the quality of wave functions may affect the results considerably and, hence, *configuration mixing* of the levels from different configurations might be desirable to incorporate. On the other hand, one has to avoid any *double counting* of levels, and which is typically not so simple to recognize internally, since different *mixings* (multiplets) results in different level energies, a very important criterion to distinguish between different levels. In practice, this makes it necessary that every (electron) *configuration* may appear only once in any of the intermediate multiplets. Moreover, configuration mixing also results in more detailed CSF expansions of all atomic levels, i.e. into a *size factor* that later appears also *quadratically* in the computation of all individual transition rates and cross sections. Further work has to show how easily more sophisticated approaches can be realized in JAC.

At present, only a `Cascade.AverageSCA()` approach has been realized, telling JAC that every electron configuration is treated independently to a single multiplet and without configuration mixing; in addition, here all continuum electrons are generated for an averaged energy between two such configurations. A few further assumptions are later recalled during the run, when the `Cascade.Computation` is performed. In the `Cascade.SCA` approach, later to be implemented, we shall allow full configuration mixing *within* each configuration as well as an improved treatment of the continuum orbitals.

In [5]:
? Cascade.AbstractCascadeApproach

`abstract type Cascade.AbstractCascadeApproach`      ... defines an abstract and a number of singleton types for the computational approach/model that is applied in order to          generate and evaluate all many-electron amplitudes of a given cascade.

  * struct AverageSCA          ... all levels in the cascade are described in single-configuration and single-CSF approximation; this (rather crude) approach      neglects all configuration-interactions and also applies just a single set of one-electron orbitals (from the least-ionized charge     state) for all considered charge states.
  * struct SCA                 ... all levels in the cascade are described in single-configuration approximation but with 'mixtures' within the configuration;     an individual mean-field is generated for each charge state and all continuum orbitals are generated for the correct transition     energy in the field of the remaining ion. Moreover, all the fine-structure transitions are calculated individually.


Keeping this information in mind, we can now define a `Cascade.Computation` as appropriate for our example above:

In [6]:
name = "Cascade after neon 1s --> 3p excitation"
grid = Radial.Grid(false)
wa   = Cascade.Computation(Cascade.Computation(); name=name, nuclearModel=Nuclear.Model(10.), grid=grid, 
                           approach=Cascade.AverageSCA(),
                           scheme=Cascade.StepwiseDecayScheme([Auger(), Radiative()], 5, 0, Shell[], Shell[]),
                           initialConfigs=[Configuration("1s^1 2s^2 2p^6 3p")] )


MethodError: MethodError: no method matching JAC.Cascade.StepwiseDecayScheme(::Array{JAC.Basics.AbstractProcess,1}, ::Int64, ::Int64, ::Array{Shell,1}, ::Array{Shell,1})
Closest candidates are:
  JAC.Cascade.StepwiseDecayScheme(::Any, ::Any, ::Any, ::Any, ::Any, !Matched::Any) at /home/fritzsch/fri/JAC.jl/src/module-Cascade.jl:46
  JAC.Cascade.StepwiseDecayScheme(::Array{JAC.Basics.AbstractProcess,1}, ::Int64, !Matched::Dict{Int64,Float64}, !Matched::Int64, ::Array{Shell,1}, !Matched::Array{Shell,1}) at /home/fritzsch/fri/JAC.jl/src/module-Cascade.jl:46

Although the output looks a bit cumbersome and may require some futher insight into the internal structure of JAC, it briefly reports about all internal settings for the given computation. Owing to the complexity of most cascade computations, it is often important to check that all details are communicated properly from the user to the program as well as *within* the program. For our example, it is sufficient to `perform(comp::Cascade.Computation)` but with the optional parameter `output=true` in order to be able to use the data for a subsequent cascade simulation. In addition, we here specify that some additional information will be printed to a summary file; in contrast to most `Atomic.Computations`, this summary file is more detailed that the standard (lo) output in order to enable the user to check that all parts of the computation were done properly (this is the *price of complexity*). 

We can run the computation by:

In [7]:
setDefaults("print summary: open", "15-cascade-computation-following-decay.sum")
wb = perform(wa; output=true)

setDefaults("print summary: close", "")

UndefVarError: UndefVarError: wa not defined

**At present, no real transition data are calculated but all rates are set arbitrarely to some constant;** of course, this will affect the subsequent simulations. Moreover, the last few lines of this output indicate that the results are (usually) also written to some external file (by using `JLD2`) from which these data can later be read-in in order to make different simulations on these data possible. Usually, the cascade tree *computations* are 2-4 orders of magnitude (computationally) more expensive than the cascade simulations.

The output above tells that, for the given example, the cascade tree is built upon 372 Auger lines and 474 photon emission lines, and with even a much larger number of transition amplitudes since several channels may contribute to each line. These numbers can easily increase to several 10.000 or even more lines for complex cascades.

As seen above, all results are also returned in the dictionary `wb` from which the subsequent simulations may start.

In [8]:
wb

UndefVarError: UndefVarError: wb not defined

We now come to the **second part of the cascade calcuations,** the `Cascade.Simulation`:

In [9]:
? Cascade.Simulation

`struct  Cascade.Simulation`  ... defines a simulation on some given cascade (data).

```
+ name            ::String                              ... Name of the simulation
+ property        ::Cascade.AbstractSimulationProperty 
    ... Property that is to be considered in this simulation of the cascade (data).
+ method          ::Cascade.AbstractSimulationMethod    
    ... Method that is used in the cascade simulation; cf. Cascade.SimulationMethod.
+ settings        ::Cascade.SimulationSettings          ... Settings for performing these simulations.
+ computationData ::Array{Dict{String,Any},1}           ... Date on which the simulations are performed
```

---

`Cascade.Simulation()`  ... constructor for an 'default' instance of a Cascade.Simulation.

---

`Cascade.Simulation(sim::Cascade.Simulation;`

```
        name=..,               properties=..,             method=..,              settings=..,     computationData=.. )
        
... constructor for re-defining the computation::Cascade.Simulation.
```


For a cascade simulation, we need to specify a `name` (string), the `properties::Array{Cascade.AbstractSimulationProperty,1}` to be simulated and the `method::Cascade.AbstractSimulationMethod`. The `properties` tell which observables are to be extracted from the given cascade tree data:

In [10]:
? Cascade.AbstractSimulationProperty

`abstract type  Cascade.AbstractSimulationProperty`       ... defines an abstract and various singleton types for the different properties that can be obtained from the simulation of          cascade data.

```
+ struct IonDistribution         
    ... simulate the 'ion distribution' as it is found after all cascade processes are completed.
+ struct FinalLevelDistribution  
    ... simulate the 'final-level distribution' as it is found after all cascade processes are completed.
+ struct PhotoAbsorption  
    ... simulate the photoabsorption cross sections for a given set of photo-excitation and ionization processes.
+ struct DecayPathes             ... determine the major 'decay pathes' of the cascade.
+ struct ElectronIntensities     ... simulate the electron-line intensities as function of electron energy.
+ struct PhotonIntensities       ... simulate  the photon-line intensities as function of electron energy. 
+ struct ElectronCoincidence     ... simulate electron-coincidence spectra.
```


At present, only the first two of these properties have been implemented in detail in JAC. For the `method`, similarly, only the **propagation of the probabilities** is currently supported:

In [11]:
? Cascade.AbstractSimulationMethod

abstract type  Cascade.AbstractSimulationMethod`       ... defines a abstract and a list of singleton data types for the properties that can be 'simulated' from a given         set of a set Cascade.AbstractData.

```
+ struct ProbPropagation     ... to propagate the (occupation) probabilites of the levels until no further changes occur.
+ struct MonteCarlo          ... to simulate the cascade decay by a Monte-Carlo approach of possible pathes (not yet considered).
+ struct RateEquations       ... to solve the cascade by a set of rate equations (not yet considered).
```


In addition, we also need to specify the `settings::Cascade.SimulationSettings` for this simulation:

In [12]:
? Cascade.SimulationSettings

`struct  Cascade.SimulationSettings`  ... defines settings for performing the simulation of some cascade (data).

```
+ minElectronEnergy   ::Float64     ... Minimum electron energy for the simulation of electron spectra.
+ maxElectronEnergy   ::Float64     ... Maximum electron energy for the simulation of electron spectra.
+ minPhotonEnergy     ::Float64     ... Minimum photon energy for the simulation of photon/absorption spectra.
+ maxPhotonEnergy     ::Float64     ... Maximum photon energy for the simulation of photon/absorption spectra.
+ initialPhotonEnergy ::Float64     ... Photon energy for which photoionization data are considered. 
+ initialOccupations  ::Array{Tuple{Int64,Float64},1}   
    ... List of one or several (tupels of) levels in the overall cascade tree together with their relative population.
```

---

`Cascade.SimulationSettings()`  ... constructor for an 'empty' instance of a Cascade.Block.


as well as the (cascade tree) data from the previous steps. Generally, data from different `Cascade.Computation()` can be combined, though care has to be taken that all these data fit properly together. Although various tests are made, it appears very difficult to recognize all mistakes that can be made by the user.

In the present example, all this input is quite simple:

In [13]:
data = [wb]
name = "Simulation of the neon 1s^-1 3p decay"

wc   = Cascade.Simulation(Cascade.Simulation(), name=name, 
                          properties=Cascade.AbstractSimulationProperty[Cascade.IonDistribution(), 
                                                                        Cascade.FinalLevelDistribution()], 
                          settings=Cascade.SimulationSettings(0., 0., 0., 0., 0., [(1, 2.0), (2, 1.0), (3, 0.5)]), 
                          computationData=data )

UndefVarError: UndefVarError: wb not defined

Apparently, we here request for the **simulation of the ion distribution** as well as **final-level distribution** of this cascade; the initial occupation of the first tree levels is given rather arbitrary and should be defined due to physical insights into the prior excitation process.

As before, we have to `perform(sim::Cascade.Simulation)`

In [14]:
wd = perform(wc; output=true)

UndefVarError: UndefVarError: wc not defined

The result for the final ion distribution simply tells us that only a tiny fraction of all atoms remains neutral due to the radiative decay back to the $1s^2 2s^2 2p^6\; ^1S_0$ ground level, while singly and doubly-charged neon is roughly the same ... at present, a consequence of the (still) unphysical specification of the amplitudes.

Already this (quite) simple example shows the **complexity of atomic cascades**; we shall develop this *branch* of JAC further, although support and a detailed discussion about the experiment, which need to be modeled, will be necessary in almost all cases.